<a href="https://colab.research.google.com/github/ipejun-ai/m5-accuracy/blob/master/m5_three_shades_of_dark_darker_magic_binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool        # Multiprocess Runs

warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
DIRPATH="/content/gdrive/My Drive/kaggle/m5-forecasting-accuracy/"
#DIRPATH="C:/Users/peiju/Documents/Study/kaggle/m5-forecasting-accuracy/"

In [0]:
########################### Helpers
#################################################################################
## Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [0]:
#nolimittation of store
def get_data():
  # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    #df = df[df['store_id']==store]

    # With memory limits we have to read 
    # lags and mean encoding features
    # separately and drop items that we don't need.
    # As our Features Grids are aligned 
    # we can use index to keep only necessary rows
    # Alignment is good for us as concat uses less memory than merge.
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]


    df4 = pd.read_pickle(SNAP).iloc[:,2:]
    df4 = df4[df4.index.isin(df.index)]

    #df5 = pd.read_pickle(EVENT_TYPE).iloc[:,2:]
    #df5 = df5[df5.index.isin(df.index)]

    df6 = pd.read_pickle(EVENT_VALUE).iloc[:,2]
    df6 = df6[df6.index.isin(df.index)]

    df7 = pd.read_pickle(LAGS_364).iloc[:,2:]
    df7 = df7[df7.index.isin(df.index)]

    df8 = pd.read_pickle(HOLIDAY).iloc[:,2:]
    df8 = df8[df8.index.isin(df.index)]

    #df5 = pd.read_pickle(EVENT_TYPE).iloc[:,2:]
    #df5 = df5[df5.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # to not reach memory limit 
    
    df = pd.concat([df, df3], axis=1)
    del df3 # to not reach memory limit 

    df = pd.concat([df, df4], axis=1)
    del df4 # to not reach memory limit 

    #df = pd.concat([df, df5], axis=1)
    #del df5 # to not reach memory limit 

    df = pd.concat([df, df6], axis=1)
    del df6 # to not reach memory limit 

    df = pd.concat([df, df7], axis=1)
    del df7 # to not reach memory limit 

    df = pd.concat([df, df8], axis=1)
    del df8 # to not reach memory limit  

   

    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features


In [0]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    df = df[df['store_id']==store]

    # With memory limits we have to read 
    # lags and mean encoding features
    # separately and drop items that we don't need.
    # As our Features Grids are aligned 
    # we can use index to keep only necessary rows
    # Alignment is good for us as concat uses less memory than merge.
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]


    df4 = pd.read_pickle(SNAP).iloc[:,2:]
    df4 = df4[df4.index.isin(df.index)]

    #df5 = pd.read_pickle(EVENT_TYPE).iloc[:,2:]
    #df5 = df5[df5.index.isin(df.index)]

    df6 = pd.read_pickle(EVENT_VALUE).iloc[:,2]
    df6 = df6[df6.index.isin(df.index)]

    df7 = pd.read_pickle(LAGS_364).iloc[:,2:]
    df7 = df7[df7.index.isin(df.index)]

    df8 = pd.read_pickle(HOLIDAY).iloc[:,2:]
    df8 = df8[df8.index.isin(df.index)]

    #df5 = pd.read_pickle(EVENT_TYPE).iloc[:,2:]
    #df5 = df5[df5.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # to not reach memory limit 
    
    df = pd.concat([df, df3], axis=1)
    del df3 # to not reach memory limit 

    df = pd.concat([df, df4], axis=1)
    del df4 # to not reach memory limit 

    #df = pd.concat([df, df5], axis=1)
    #del df5 # to not reach memory limit 

    df = pd.concat([df, df6], axis=1)
    del df6 # to not reach memory limit 

    df = pd.concat([df, df7], axis=1)
    del df7 # to not reach memory limit 

    df = pd.concat([df, df8], axis=1)
    del df8 # to not reach memory limit  

   

    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [0]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
                    #'boosting_type': 'gbdt',
                    'boosting': 'dart',          # dart (drop out trees) often performs better
                    'application': 'binary',
                   
                    'metric': 'auc',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    #'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 

# Let's look closer on params

## 'boosting_type': 'gbdt'
# we have 'goss' option for faster training
# but it normally leads to underfit.
# Also there is good 'dart' mode
# but it takes forever to train
# and model performance depends 
# a lot on random factor 
# https://www.kaggle.com/c/home-credit-default-risk/discussion/60921

## 'objective': 'tweedie'
# Tweedie Gradient Boosting for Extremely
# Unbalanced Zero-inflated Data
# https://arxiv.org/pdf/1811.10192.pdf
# and many more articles about tweediie
#
# Strange (for me) but Tweedie is close in results
# to my own ugly loss.
# My advice here - make OWN LOSS function
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143070
# I think many of you already using it (after poisson kernel appeared) 
# (kagglers are very good with "params" testing and tuning).
# Try to figure out why Tweedie works.
# probably it will show you new features options
# or data transformation (Target transformation?).

## 'tweedie_variance_power': 1.1
# default = 1.5
# set this closer to 2 to shift towards a Gamma distribution
# set this closer to 1 to shift towards a Poisson distribution
# my CV shows 1.1 is optimal 
# but you can make your own choice

## 'metric': 'rmse'
# Doesn't mean anything to us
# as competition metric is different
# and we don't use early stoppings here.
# So rmse serves just for general 
# model performance overview.
# Also we use "fake" validation set
# (as it makes part of the training set)
# so even general rmse score doesn't mean anything))
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834

## 'subsample': 0.5
# Serves to fight with overfit
# this will randomly select part of data without resampling
# Chosen by CV (my CV can be wrong!)
# Next kernel will be about CV

##'subsample_freq': 1
# frequency for bagging
# default value - seems ok

## 'learning_rate': 0.03
# Chosen by CV
# Smaller - longer training
# but there is an option to stop 
# in "local minimum"
# Bigger - faster training
# but there is a chance to
# not find "global minimum" minimum

## 'num_leaves': 2**11-1
## 'min_data_in_leaf': 2**12-1
# Force model to use more features
# We need it to reduce "recursive"
# error impact.
# Also it leads to overfit
# that's why we use small 
# 'max_bin': 100

## l1, l2 regularizations
# https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c
# Good tiny explanation
# l2 can work with bigger num_leaves
# but my CV doesn't show boost
                    
## 'n_estimators': 1400
# CV shows that there should be
# different values for each state/store.
# Current value was chosen 
# for general purpose.
# As we don't use any early stopings
# careful to not overfit Public LB.

##'feature_fraction': 0.5
# LightGBM will randomly select 
# part of features on each iteration (tree).
# We have maaaany features
# and many of them are "duplicates"
# and many just "noise"
# good values here - 0.5-0.7 (by CV)

## 'boost_from_average': False
# There is some "problem"
# to code boost_from_average for 
# custom loss
# 'True' makes training faster
# BUT carefull use it
# https://github.com/microsoft/LightGBM/issues/1514
# not our case but good to know cons

# Vars & Path

In [0]:
########################### Vars
#################################################################################
VER = 22                          # Our model version
SEED = 42                        # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
P_HORIZON   = 28
END_TRAIN   = 1913-P_HORIZON             # End day of our train set
                 # Prediction horizon
USE_AUX     = False           # Use or not pretrained models
USE_PKL     =False

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
remove_features = ['id','state_id'
                    #,'store_id',
                   'date','wm_yr_wk','d','snap_CA','snap_TX','snap_WI'
                   ,"event_name_1",
                   #"event_type_1","event_name_2","event_type_2",
                   TARGET]
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

##selected feature by boruta
selected_feature=['item_id', 'dept_id', 'release', 'sell_price', 'price_max', 'price_min',
       'price_std', 'price_mean', 'price_norm', 'item_nunique',
       'price_momentum_m', 'price_momentum_y', 'event_type_1', 'tm_d', 'tm_w',
       'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_dept_id_mean',
       'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std',
       'sales_lag_28', 'sales_lag_35', 'rolling_mean_7', 'rolling_std_7',
       'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30',
       'rolling_std_30', 'rolling_mean_60', 'rolling_std_60',
       'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7',
       'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30',
       'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7',
       'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30',
       'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7',
       'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30',
       'rolling_mean_tmp_14_60', 'snap', 'Event_total',
       'sales_lag_28_364', 'rolling_mean_7_364',
       'rolling_std_7_364', 'rolling_mean_14_364', 'rolling_std_14_364',
       'rolling_mean_30_364', 'rolling_std_30_364', 'holiday',
       'rolling_holiday_mean_7', 'rolling_holiday_mean_14',
       'rolling_holiday_mean_30', 'rolling_holiday_mean_60',
       'rolling_holiday_mean_180']

#PATHS for Features
ORIGINAL = DIRPATH
BASE     = DIRPATH+'/output/m5-simple-fe/grid_part_1.pkl'
PRICE    = DIRPATH+'/output/m5-simple-fe/grid_part_2.pkl'
CALENDAR = DIRPATH+'/output/m5-simple-fe/grid_part_3.pkl'
LAGS     = DIRPATH+'/output/m5-lags-features/lags_df_28.pkl'
MEAN_ENC = DIRPATH+'/output/m5-custom-features/mean_encoding_df.pkl'
FINAL_SALES = DIRPATH+'/output/m5-custom-features/finalsales_df.pkl'
SNAP      =  DIRPATH+'/output/m5-custom-features/snap_df.pkl'
EVENT_TYPE      =  DIRPATH+'/output/m5-custom-features/event_type.pkl'
EVENT_VALUE      =  DIRPATH+'/output/m5-custom-features/event_value.pkl'
LAGS_364=  DIRPATH+'/output/m5-custom-features/lags_df_364.pkl'
HOLIDAY=DIRPATH+'/output/m5-custom-features/holiday_mean.pkl'


#PATHS for Features
"""
ORIGINAL = DIRPATH
BASE     = DIRPATH+'/pickle/grid_part_1.pkl'
PRICE    = DIRPATH+'/pickle/grid_part_2.pkl'
CALENDAR = DIRPATH+'/pickle/grid_part_3.pkl'
LAGS     = DIRPATH+'//pickle//lags_df_28.pkl'
MEAN_ENC = DIRPATH+'//pickle//mean_encoding_df.pkl'
FINAL_SALES = DIRPATH+'//pickle//finalsales_df.pkl'
SNAP      =  DIRPATH+'//pickle//snap_df.pkl'
EVENT_TYPE      =  DIRPATH+'//pickle//event_type.pkl'
EVENT_VALUE      =  DIRPATH+'//pickle//event_value.pkl'
LAGS_364=  DIRPATH+'//pickle//lags_df_364.pkl'
HOLIDAY=DIRPATH+'//pickle//holiday_mean.pkl'
"""
# AUX(pretrained) Models paths
AUX_MODELS = DIRPATH+'/output/m5-three-shades-of-dark-darker-magic_binary/'

SAVE_DIR = DIRPATH+'/output/m5-three-shades-of-dark-darker-magic_binary/'


#STORES ids
STORES_IDS = pd.read_csv(ORIGINAL+'sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())


#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [0]:
########################### Aux Models
# If you don't want to wait hours and hours
# to have result you can train each store 
# in separate kernel and then just join result.

# If we want to use pretrained models we can 
## skip training 
## (in our case do dummy training
##  to show that we are good with memory
##  and you can safely use this (all kernel) code)
if USE_AUX:
    lgb_params['n_estimators'] = 2
    
# Here is some 'logs' that can compare
#Train CA_1
#[100]	valid_0's rmse: 2.02289
#[200]	valid_0's rmse: 2.0017
#[300]	valid_0's rmse: 1.99239
#[400]	valid_0's rmse: 1.98471
#[500]	valid_0's rmse: 1.97923
#[600]	valid_0's rmse: 1.97284
#[700]	valid_0's rmse: 1.96763
#[800]	valid_0's rmse: 1.9624
#[900]	valid_0's rmse: 1.95673
#[1000]	valid_0's rmse: 1.95201
#[1100]	valid_0's rmse: 1.9476
#[1200]	valid_0's rmse: 1.9434
#[1300]	valid_0's rmse: 1.9392
#[1400]	valid_0's rmse: 1.93446

#Train CA_2
#[100]	valid_0's rmse: 1.88949
#[200]	valid_0's rmse: 1.84767
#[300]	valid_0's rmse: 1.83653
#[400]	valid_0's rmse: 1.82909
#[500]	valid_0's rmse: 1.82265
#[600]	valid_0's rmse: 1.81725
#[700]	valid_0's rmse: 1.81252
#[800]	valid_0's rmse: 1.80736
#[900]	valid_0's rmse: 1.80242
#[1000]	valid_0's rmse: 1.79821
#[1100]	valid_0's rmse: 1.794
#[1200]	valid_0's rmse: 1.78973
#[1300]	valid_0's rmse: 1.78552
#[1400]	valid_0's rmse: 1.78158

In [10]:
"""
if USE_PKL:
  grid_df_all=pd.read_pickle(AUX_MODELS+'grid_df_all_v'+str(VER)+'.pkl') 
  features=pd.read_csv(AUX_MODELS+'feature_v'+str(VER)+'.csv',header=None)
  features_columns=list(features[0])
else:
  grid_df_all, features_columns = get_data()
  grid_df_all.to_pickle(AUX_MODELS+'grid_df_all_v'+str(VER)+'.pkl')
  imp=pd.DataFrame({'feature': features_columns})
  imp.to_csv(AUX_MODELS+'feature_v'+str(VER)+'.csv', index=False)
"""

"\nif USE_PKL:\n  grid_df_all=pd.read_pickle(AUX_MODELS+'grid_df_all_v'+str(VER)+'.pkl') \n  features=pd.read_csv(AUX_MODELS+'feature_v'+str(VER)+'.csv',header=None)\n  features_columns=list(features[0])\nelse:\n  grid_df_all, features_columns = get_data()\n  grid_df_all.to_pickle(AUX_MODELS+'grid_df_all_v'+str(VER)+'.pkl')\n  imp=pd.DataFrame({'feature': features_columns})\n  imp.to_csv(AUX_MODELS+'feature_v'+str(VER)+'.csv', index=False)\n"

# Train

In [11]:
########################### Train Models
#################################################################################
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # Get grid for current store
    grid_df, features_columns = get_data_by_store(store_id)
    #change_binary
    grid_df.loc[grid_df["sales"]>0,"sales"]=1
    grid_df.loc[grid_df["sales"]==0,"sales"]=0

    #grid_df=grid_df_all[grid_df_all['store_id']==store_id]
    #grid_df=grid_df.drop("store_id",axis=1)
    ##borutaで選択した特徴量に変換
    features_columns=[x for x in features_columns if x in selected_feature]
    
    # Masks for 
    # Train (All data less than 1913)
    # "Validation" (Last 28 days - not real validatio set)
    # Test (All data greater than 1913 day, 
    #       with some gap for recursive features)
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = (grid_df['d']<=(END_TRAIN+P_HORIZON))&(grid_df['d']>(END_TRAIN))
    preds_mask = grid_df['d']>(END_TRAIN-100)
    
    # Apply masks and save lgb dataset as bin
    # to reduce memory spikes during dtype convertations
    # https://github.com/Microsoft/LightGBM/issues/1032
    # "To avoid any conversions, you should always use np.float32"
    # or save to bin before start training
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                       label=grid_df[train_mask][TARGET])
    train_data.save_binary('train_data.bin')
    train_data = lgb.Dataset('train_data.bin')
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])
    
    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+store_id+'.pkl')
    del grid_df
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )
    
    # Save model - it's not real '.bin' but a pickle file
    # estimator = lgb.Booster(model_file='model.txt')
    # can only predict with the best iteration (or the saving iteration)
    # pickle.dump gives us more flexibility
    # like estimator.predict(TEST, num_iteration=100)
    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    if USE_AUX==False:
      model_name = AUX_MODELS+'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
      pickle.dump(estimator, open(model_name, 'wb'))

    # Remove temporary files and objects 
    # to free some hdd space and ram memory
    !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()
    
    # "Keep" models features for predictions
    MODEL_FEATURES = features_columns

Train CA_1
[100]	valid_0's auc: 0.825178
Train CA_2
[100]	valid_0's auc: 0.810465
Train CA_3
[100]	valid_0's auc: 0.829121
Train CA_4
[100]	valid_0's auc: 0.797963
Train TX_1
[100]	valid_0's auc: 0.819183
Train TX_2
[100]	valid_0's auc: 0.808797
Train TX_3
[100]	valid_0's auc: 0.822968
Train WI_1
[100]	valid_0's auc: 0.820104
Train WI_2
[100]	valid_0's auc: 0.846052
Train WI_3
[100]	valid_0's auc: 0.824103


# Evaluation

In [0]:
############save importance
for store_id in STORES_IDS:
  model_path = AUX_MODELS + 'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
  estimator = pickle.load(open(model_path, 'rb'))
  
  if store_id=="CA_1":
    imp=pd.DataFrame({'feature': features_columns,store_id:estimator.feature_importance()})
  else:
    tmp=pd.DataFrame({'feature': features_columns,store_id:estimator.feature_importance()})
    imp=pd.merge(imp,tmp,how="left",on="feature")

imp.index=imp["feature"]
imp=imp.drop("feature",axis=1)
imp["mean"]=imp.mean(axis=1)
imp.to_csv(AUX_MODELS+"importance_v"+str(VER)+".csv")

In [0]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()
import gc

from sklearn import preprocessing
import lightgbm as lgb

from typing import Union
from tqdm.notebook import tqdm_notebook as tqdm

In [15]:
########################### Predict Train & Valid
#################################################################################

# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day
for PREDICT_DAY in range(1,29):    
#for PREDICT_DAY in range(1,2):    
 
    print('Predict | Day:', PREDICT_DAY,END_TRAIN+PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
    grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)
        
    for store_id in STORES_IDS:
        
        # Read all our models and make predictions
        # for each day/store pairs
        model_path = AUX_MODELS + 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 
        if USE_AUX:
            model_path = model_path
        
        estimator = pickle.load(open(model_path, 'rb'))
        
        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id
        
        mask = (day_mask)&(store_mask)
        #base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
        base_test.loc[mask,TARGET]=estimator.predict(grid_df[mask][MODEL_FEATURES])
    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)

Predict | Day: 1 1886
##########  0.66 min round |  0.66 min total |  11159.71 day sales |
Predict | Day: 2 1887
##########  0.63 min round |  1.29 min total |  11402.00 day sales |
Predict | Day: 3 1888
##########  0.61 min round |  1.89 min total |  11455.79 day sales |
Predict | Day: 4 1889
##########  0.61 min round |  2.50 min total |  11379.37 day sales |
Predict | Day: 5 1890
##########  0.62 min round |  3.13 min total |  11874.62 day sales |
Predict | Day: 6 1891
##########  0.62 min round |  3.74 min total |  12726.47 day sales |
Predict | Day: 7 1892
##########  0.63 min round |  4.37 min total |  12732.15 day sales |
Predict | Day: 8 1893
##########  0.62 min round |  4.99 min total |  11554.57 day sales |
Predict | Day: 9 1894
##########  0.61 min round |  5.60 min total |  11598.49 day sales |
Predict | Day: 10 1895
##########  0.61 min round |  6.21 min total |  11564.79 day sales |
Predict | Day: 11 1896
##########  0.61 min round |  6.82 min total |  11512.35 day sales

In [0]:
#reload data
d_dtypes = {}
for i in range(1914):
    d_dtypes[f'd_{i}'] = np.int32
sales = pd.read_csv(DIRPATH + 'sales_train_validation.csv',
                    dtype=d_dtypes)

# changing wide format to long format for model training
d = ['d_' + str(i) for i in range(1802,1914)]
sales_mlt = pd.melt(sales, id_vars=['item_id','dept_id','cat_id','store_id',
                                    'state_id'], value_vars=d)
sales_mlt = sales_mlt.rename(columns={'variable':'d', 'value':'sales'})

calendar = pd.read_csv(DIRPATH + 'calendar.csv',
                       dtype={'wm_yr_wk': np.int32, 'wday': np.int32, 
                              'month': np.int32, 'year': np.int32, 
                              'snap_CA': np.int32, 'snap_TX': np.int32,
                              'snap_WI': np.int32})

# subsetting calender by traning period
calendar = calendar.loc[calendar.d.apply(lambda x: int(x[2:])) \
                        >= int(sales_mlt.d[0][2:]), :]

prices = pd.read_csv(DIRPATH + 'sell_prices.csv',
                          dtype={'wm_yr_wk': np.int32, 
                                 'sell_price': np.float32})

train_df = sales.iloc[:, :-28]
valid_df = sales.iloc[:, -28:]

#evaluator = WRMSSEEvaluator(train_df, valid_df, calendar, prices)

#sort
all_preds=pd.merge(train_df["id"],all_preds,on="id",how="left")
all_preds=all_preds.drop("id",axis=1)
all_preds.to_csv(AUX_MODELS+'validation_v'+str(VER)+'.csv', index=False)


In [56]:
sales_tmp=sales.iloc[:,len(sales.columns)-28:]
sales_tmp[sales_tmp>=1]=1
sales_tmp

from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np

# FPR, TPR(, しきい値) を算出
fpr, tpr, thresholds = metrics.roc_curve(np.array(sales_tmp).flatten(),np.array(all_preds).flatten(),pos_label=0)

# ついでにAUCも
auc = metrics.auc(fpr, tpr)
print(auc,thresholds)

# ROC曲線をプロット
plt.plot(fpr, tpr, label='ROC curve (area = %.2f)'%auc)
plt.legend()
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)

ValueError: ignored

In [28]:
#####confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score

y_true=np.array(sales_tmp).flatten()
y_pred=np.array(all_preds).flatten()
cm=confusion_matrix(y_true, y_pred>0.3)
tn, fp, fn, tp = cm.flatten()

precision_score(y_true, y_pred>0.3,pos_label=0)


0.8841016521415075

In [50]:
np.arange(0.1,1.0, 0.05)

array([0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55, 0.6 ,
       0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95])

In [0]:
df=pd.DataFrame({'thr':np.arange(0.1,0.6, 0.03)})
df['zero_precision'] = df.apply(lambda row: precision_score(y_true, y_pred >= row["thr"],pos_label=0), axis=1)
df["tn"]=df.apply(lambda row: confusion_matrix(y_true, y_pred >= row["thr"]).flatten()[0], axis=1)
df["fp"]=df.apply(lambda row: confusion_matrix(y_true, y_pred >= row["thr"]).flatten()[1], axis=1)
df["fn"]=df.apply(lambda row: confusion_matrix(y_true, y_pred >= row["thr"]).flatten()[2], axis=1)
df["tp"]=df.apply(lambda row: confusion_matrix(y_true, y_pred >= row["thr"]).flatten()[3], axis=1)
df.to_csv(AUX_MODELS+"confusion_matrix_v"+str(VER)+".csv")
#df['FPR'] = df.apply(lambda row: fpr_score(y_true, y_score >= row['score']), axis=1)

In [68]:
df

,thr,zero_precision,tn,fp,fn,tp
0,0.10,0.000000,0,480460,0,373260
1,0.13,0.000000,0,480460,0,373260
2,0.16,0.000000,0,480460,0,373260
3,0.19,0.990120,1403,479057,14,373246
4,0.22,0.961070,6937,473523,281,372979
5,0.25,0.932462,42676,437784,3091,370169
6,0.28,0.903628,96306,384154,10271,362989
7,0.31,0.874047,155097,325363,22350,350910
8,0.34,0.844878,217796,262664,39988,333272
9,0.37,0.810441,283061,197399,66207,307053


# Prediction

In [35]:
########################### Predict
#################################################################################

# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day
for PREDICT_DAY in range(1,29):    
#for PREDICT_DAY in range(1,2):    
 
    print('Predict | Day:', PREDICT_DAY,END_TRAIN+PREDICT_DAY+P_HORIZON)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
    grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)
        
    for store_id in STORES_IDS:
        
        # Read all our models and make predictions
        # for each day/store pairs
        model_path = AUX_MODELS + 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 
        if USE_AUX:
            model_path = model_path
        
        estimator = pickle.load(open(model_path, 'rb'))
        
        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY+P_HORIZON)
        store_mask = base_test['store_id']==store_id
        
        mask = (day_mask)&(store_mask)
        #base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
        base_test.loc[mask,TARGET]=estimator.predict(grid_df[mask][MODEL_FEATURES])
    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)

Predict | Day: 1 1914
##########  0.63 min round |  0.63 min total |  11204.35 day sales |
Predict | Day: 2 1915
##########  0.63 min round |  1.27 min total |  11342.18 day sales |
Predict | Day: 3 1916
##########  0.60 min round |  1.87 min total |  11410.05 day sales |
Predict | Day: 4 1917
##########  0.61 min round |  2.48 min total |  11325.73 day sales |
Predict | Day: 5 1918
##########  0.62 min round |  3.10 min total |  11602.93 day sales |
Predict | Day: 6 1919
##########  0.62 min round |  3.71 min total |  12374.80 day sales |
Predict | Day: 7 1920
##########  0.60 min round |  4.31 min total |  12498.00 day sales |
Predict | Day: 8 1921
##########  0.61 min round |  4.92 min total |  11563.99 day sales |
Predict | Day: 9 1922
##########  0.61 min round |  5.53 min total |  11570.04 day sales |
Predict | Day: 10 1923
##########  0.61 min round |  6.14 min total |  11438.47 day sales |
Predict | Day: 11 1924
##########  0.61 min round |  6.75 min total |  11491.84 day sales

In [0]:
########################### Export
#################################################################################
# Reading competition sample submission and
# merging our predictions
# As we have predictions only for "_validation" data
# we need to do fillna() for "_evaluation" items
submission = pd.read_csv(ORIGINAL+'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv(AUX_MODELS+'submission_v'+str(VER)+'.csv', index=False)
submission.to_csv('submission_v'+str(VER)+'.csv', index=False)



In [0]:
# Summary

# Of course here is no magic at all.
# No "Novel" features and no brilliant ideas.
# We just carefully joined all
# our previous fe work and created a model.

# Also!
# In my opinion this strategy is a "dead end".
# Overfits a lot LB and with 1 final submission 
# you have no option to risk.


# Improvement should come from:
# Loss function
# Data representation
# Stable CV
# Good features reduction strategy
# Predictions stabilization with NN
# Trend prediction
# Real zero sales detection/classification


# Good kernels references 
## (the order is random and the list is not complete):
# https://www.kaggle.com/ragnar123/simple-lgbm-groupkfold-cv
# https://www.kaggle.com/jpmiller/grouping-items-by-stockout-pattern
# https://www.kaggle.com/headsortails/back-to-predict-the-future-interactive-m5-eda
# https://www.kaggle.com/sibmike/m5-out-of-stock-feature
# https://www.kaggle.com/mayer79/m5-forecast-attack-of-the-data-table
# https://www.kaggle.com/yassinealouini/seq2seq
# https://www.kaggle.com/kailex/m5-forecaster-v2
# https://www.kaggle.com/aerdem4/m5-lofo-importance-on-gpu-via-rapids-xgboost


# Features were created in these kernels:
## 
# Mean encodings and PCA options
# https://www.kaggle.com/kyakovlev/m5-custom-features
##
# Lags and rolling lags
# https://www.kaggle.com/kyakovlev/m5-lags-features
##
# Base Grid and base features (calendar/price/etc)
# https://www.kaggle.com/kyakovlev/m5-simple-fe


# Personal request
# Please don't upvote any ensemble and copypaste kernels
## The worst case is ensemble without any analyse.
## The best choice - just ignore it.
## I would like to see more kernels with interesting and original approaches.
## Don't feed copypasters with upvotes.

## It doesn't mean that you should not fork and improve others kernels
## but I would like to see params and code tuning based on some CV and analyse
## and not only on LB probing.
## Small changes could be shared in comments and authors can improve their kernel.

## Feel free to criticize this kernel as my knowlege is very limited
## and I can be wrong in code and descriptions. 
## Thank you.

In [0]:
imp=pd.DataFrame({'feature': features_columns,

 'importance':estimator.feature_importance()}).sort_values('importance',

 ascending=False)
imp.to_csv(AUX_MODELS+'importance_v'+str(VER)+'.csv', index=False)

In [0]:
for store_id in STORES_IDS:
  model_path = AUX_MODELS + 'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
  estimator = pickle.load(open(model_path, 'rb'))
  
  if store_id=="CA_1":
    imp=pd.DataFrame({'feature': features_columns,store_id:estimator.feature_importance()})
  else:
    tmp=pd.DataFrame({'feature': features_columns,store_id:estimator.feature_importance()})
    imp=pd.merge(imp,tmp,how="left",on="feature")

imp.index=imp["feature"]
imp=imp.drop("feature",axis=1)
imp["mean"]=imp.mean(axis=1)
imp.to_csv(AUX_MODELS+"importance_v"+str(VER)+".csv")


        
  

feature
item_id                     619640
dept_id                      93551
release                      70404
sell_price                   78391
price_max                    72620
                             ...  
rolling_holiday_mean_7       60800
rolling_holiday_mean_14      73897
rolling_holiday_mean_30      93357
rolling_holiday_mean_60     121183
rolling_holiday_mean_180    155797
Length: 63, dtype: int64